# 1) 데이터 준비와 전처리

In [156]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [157]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [158]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 데이터 분석

In [159]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [160]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [161]:
# 가장 인기 있는 영화 30개(인기순)

# ratings, movies 데이터 합치기
data = pd.merge(movies, ratings)
data.head()

,movie_id,title,genre,user_id,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [162]:
# 사용할 컬럼만 남기기
using_cols = ['title', 'rating', 'user_id']
data = data[using_cols]
data.head()

,title,rating,user_id
0,Toy Story (1995),5,1
1,Toy Story (1995),4,6
2,Toy Story (1995),4,8
3,Toy Story (1995),5,9
4,Toy Story (1995),5,10


In [163]:
# 영화이름 대문자 일괄 소문자로 변환
data['title'] = data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head()

,title,rating,user_id
0,toy story (1995),5,1
1,toy story (1995),4,6
2,toy story (1995),4,8
3,toy story (1995),5,9
4,toy story (1995),5,10


In [164]:
# 가장 인기 있는 영화 30개(인기순)
top_movie_count = data.groupby('title')['user_id'].count()
top_movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

# 3) 내가 선호하는 영화 5가지 rating에 추가

In [165]:
my_favorite = ['jurassic park (1993)', 'matrix, the (1999)', 'back to the future (1985)', 'shawshank redemption, the (1994)', 'forrest gump (1994)']

# 'dreamb'이라는 user_id가 위 영화를 5점씩 줬다고 가정
my_playlist = pd.DataFrame({'user_id': ['dreamb']*5, 'title': my_favorite, 'rating':[5]*5})

if not data.isin({'user_id':['dreamb']})['user_id'].any():  # user_id에 'dreamb'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,title,rating,user_id
836473,"contender, the (2000)",3,5682
836474,"contender, the (2000)",4,5812
836475,"contender, the (2000)",3,5831
836476,"contender, the (2000)",4,5837
836477,"contender, the (2000)",4,5998
0,jurassic park (1993),5,dreamb
1,"matrix, the (1999)",5,dreamb
2,back to the future (1985),5,dreamb
3,"shawshank redemption, the (1994)",5,dreamb
4,forrest gump (1994),5,dreamb


In [166]:
# rating 컬럼의 이름을 count로 바꿉니다.
#data.rename(columns={'rating':'count'}, inplace=True)

In [167]:
#data['count']

# 4) CSR Matrix

In [168]:
# 모델에 활용하기 위한 전처리

# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 영화 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [169]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['dreamb'])  
print(title_to_idx['forrest gump (1994)'])

6039
342


In [170]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,title,rating,user_id
0,0,5,0
1,0,4,1
2,0,4,2
3,0,5,3
4,0,5,4
...,...,...,...
0,462,5,6039
1,2325,5,6039
2,1152,5,6039
3,305,5,6039


In [171]:
# CSR Matrix 만들기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델 구성 및 훈련

In [172]:
from implicit.als import AlternatingLeastSquares
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 / 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 / 3. use_gpu : GPU를 사용할 것인지 / 4. iterations : epochs

In [173]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=250, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [174]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [175]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [58]:
dreamb, matrix_the = user_to_idx['dreamb'], title_to_idx['matrix, the (1999)']
dreamb_vector, matrix_the_vector = als_model.user_factors[dreamb], als_model.item_factors[matrix_the]

In [59]:
dreamb_vector

array([-1.64918229e-01,  5.73568285e-01, -2.82109946e-01, -4.23898965e-01,
        9.37512368e-02, -1.85296878e-01,  5.76056898e-01, -5.49234301e-02,
        3.31508905e-01, -2.72613049e-01, -1.64384082e-01, -2.30439946e-01,
        3.72562408e-01,  1.06371410e-01,  2.69349843e-01, -3.95936012e-01,
        2.99788535e-01, -5.80991089e-01,  2.46397302e-01, -2.17045080e-02,
        4.74138498e-01,  1.33611605e-01, -6.42029047e-02, -7.60823488e-02,
       -2.81636059e-01,  5.09896874e-01,  1.79564342e-01,  5.93790822e-02,
       -1.84923425e-01, -8.12927540e-03,  2.53449497e-03,  1.61126554e-01,
        8.35144639e-01,  1.22708464e-02, -2.38546833e-01,  5.93397468e-02,
        5.45891702e-01,  1.01919763e-01,  6.22202337e-01,  2.54703611e-01,
       -2.91147586e-02,  7.31506516e-05, -7.39881516e-01,  4.33835894e-01,
       -3.52286935e-01, -3.13708186e-01, -2.13472918e-01, -6.79989904e-02,
        2.29355529e-01,  1.08870111e-01,  5.15857376e-02, -2.25754753e-01,
        2.58464944e-02,  

In [60]:
matrix_the_vector

array([ 2.65543144e-02,  4.91694547e-03,  1.07764583e-02, -5.16869454e-03,
        1.94175038e-02, -1.94793157e-02,  2.05036905e-02, -4.83246706e-03,
        2.75143981e-02, -4.05156193e-03,  1.46290679e-02, -1.09284911e-02,
        2.68365815e-02,  2.82523804e-03, -3.74334236e-03, -2.98567046e-03,
        1.49057596e-03, -1.49319237e-02,  1.67851131e-02,  1.92031860e-02,
        2.21098326e-02,  3.27741727e-02,  3.55037511e-03,  3.52190733e-02,
        1.48184942e-02,  1.89499948e-02,  2.85137501e-02,  4.43932638e-02,
       -1.36518832e-02,  1.08147142e-02,  1.50119411e-02,  1.25162639e-02,
        3.61734629e-02,  5.98386070e-03, -2.30830349e-02, -9.46708675e-03,
        6.02251245e-03, -3.71994032e-03, -6.91301515e-03,  2.51683071e-02,
        1.06905634e-02,  3.53011303e-02, -9.57863778e-03,  2.25681048e-02,
       -1.92662046e-04, -2.89627574e-02, -2.23441385e-02, -1.68764386e-02,
        9.09156399e-04,  2.32332200e-02,  1.42558869e-02, -1.21305902e-02,
        1.63106881e-02,  

In [61]:
# dreamb와 matrix_the를 내적
np.dot(dreamb_vector, matrix_the_vector)

0.75349844

In [62]:
# dreamb와 forrest_gump를 내적
forrest_gump = title_to_idx['forrest gump (1994)']
forrest_gump_vector = als_model.item_factors[forrest_gump]
np.dot(dreamb_vector, forrest_gump_vector)

0.8587298

# 7) 내가 좋아하는 영화와 비슷한 영화 추천

In [67]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화 찾기

# title의 id를 다시 title의 이름으로 매핑
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_movie(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [68]:
get_similar_movie('jurassic park (1993)')

['jurassic park (1993)',
 'men in black (1997)',
 'terminator 2: judgment day (1991)',
 'lost world: jurassic park, the (1997)',
 'character (karakter) (1997)',
 'mother night (1996)',
 'total recall (1990)',
 'little odessa (1994)',
 'bewegte mann, der (1994)',
 "greaser's palace (1972)"]

In [69]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'fugitive, the (1993)',
 'fifth element, the (1997)',
 'total recall (1990)',
 'face/off (1997)',
 'surviving picasso (1996)',
 'terminator, the (1984)',
 "mummy's ghost, the (1944)",
 'beefcake (1999)']

# 8) 내가 가장 좋아할 만한 영화 추천

In [78]:
# AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 좋아할 만한 영화 추천

user = user_to_idx['dreamb']
# recommend에서는 user*item CSR Matrix를 받습니다.
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(569, 0.3777329),
 (1419, 0.29069903),
 (573, 0.28006768),
 (1147, 0.24354617),
 (439, 0.2097311),
 (1532, 0.20585713),
 (106, 0.19860813),
 (2507, 0.17826758),
 (722, 0.17636447),
 (1082, 0.16099185),
 (1810, 0.16053948),
 (3474, 0.15704742),
 (587, 0.15491666),
 (2600, 0.14965357),
 (1028, 0.14958562),
 (523, 0.14790553),
 (1391, 0.14516205),
 (249, 0.13759197),
 (842, 0.13566369),
 (49, 0.1348353)]

In [79]:
# 추천하는 영화

[idx_to_title[i[0]] for i in title_recommended]

['terminator 2: judgment day (1991)',
 'men in black (1997)',
 'silence of the lambs, the (1991)',
 'groundhog day (1993)',
 'fugitive, the (1993)',
 'good will hunting (1997)',
 'braveheart (1995)',
 'sixth sense, the (1999)',
 'independence day (id4) (1996)',
 'raiders of the lost ark (1981)',
 'saving private ryan (1998)',
 'x-men (2000)',
 'fargo (1996)',
 'american beauty (1999)',
 'escape from new york (1981)',
 'blade runner (1982)',
 'lost world: jurassic park, the (1997)',
 'star wars: episode iv - a new hope (1977)',
 'citizen kane (1941)',
 'usual suspects, the (1995)']

In [80]:
# 이 추천에 기여한 정도를 확인

terminator_2 = title_to_idx['terminator 2: judgment day (1991)']
explain = als_model.explain(user, csr_data, itemid=terminator_2)

[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('matrix, the (1999)', 0.24726991104469637),
 ('jurassic park (1993)', 0.13674532632951691),
 ('shawshank redemption, the (1994)', 0.024430559144623852),
 ('back to the future (1985)', 0.0007161971783517568),
 ('forrest gump (1994)', -0.0363531892714886)]

'terminator 2: judgment day (1991)'를 추천하는 데 기여한 정도를 확인해 보면, 'matrix, the (1999)'와 'jurassic park (1993)'가 가장 크게 기여했다.

이 영화들의 장르를 보면,
- 공통적으로 Action과 Sci-Fi가 들어가 있는 것을 확인할 수 있다.
- 추천된 영화인 'terminator 2: judgment day (1991)'와 기여도가 가장 큰 'matrix, the (1999)'는 장르가 완전히 일치한다.
- ==> 이러한 점(장르의 일치여부)이 큰 기여도의 원인이라는 것을 유추할 수 있다.

In [135]:
tg_target_df = pd.DataFrame(movies, columns=['title', 'genre'])

tg_targets = ['terminator 2: judgment day (1991)', 'matrix, the (1999)', 'jurassic park (1993)']

tg_target_df2 = tg_target_df.loc[tg_target_df['title'].isin(tg_targets)]
tg_target_df2.head()

,title,genre
476,jurassic park (1993),Action|Adventure|Sci-Fi
585,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
2502,"matrix, the (1999)",Action|Sci-Fi|Thriller


반면, 기여도가 낮은 영화의 장르를 보면,
- shawshank redemption, the (1994)	Drama
- back to the future (1985)	 Comedy|Sci-Fi
- forrest gump (1994)   Comedy|Romance|War
- ==> 공통된 장르가 있는 'back to the future (1985)'의 기여도가 공통된 장르가 없는 'shawshank redemption, the (1994)'보다 낮게 나왔다.
- ==> 공통된 장르가 아예 없는 'forrest gump (1994)'는 마이너스 값의 기여도를 가지기 때문에 의미가 없다.
- ====> 따라서 장르 이외의 요소가 기여도에 영향을 끼쳤음을 유추할 수 있다.

In [137]:
df = pd.DataFrame(movies, columns=['title', 'genre'])

targets = ['shawshank redemption, the (1994)', 'back to the future (1985)', 'forrest gump (1994)']

df2 = df.loc[df['title'].isin(targets)]
df2.head()

,title,genre
315,"shawshank redemption, the (1994)",Drama
352,forrest gump (1994),Comedy|Romance|War
1250,back to the future (1985),Comedy|Sci-Fi


In [153]:
# 기여도의 다른 원인을 찾기 위해 rating을 비교해 보기 위한 준비
title_rating_df = pd.DataFrame(data, columns=['title', 'rating'])


title_rating_targets = ['shawshank redemption, the (1994)', 'back to the future (1985)', 'forrest gump (1994)']

title_rating_df2 = title_rating_df.loc[title_rating_df['title'].isin(title_rating_targets)]
title_rating_df2.head()

,title,rating
69600,"shawshank redemption, the (1994)",5
69601,"shawshank redemption, the (1994)",3
69602,"shawshank redemption, the (1994)",5
69603,"shawshank redemption, the (1994)",4
69604,"shawshank redemption, the (1994)",5


In [154]:
# title 별로 rating 값 평균내기
tr_df2 = title_rating_df2.groupby('title')['rating'].agg(**{'rating':'mean'}).reset_index()
tr_df2

,title,rating
0,back to the future (1985),4.097967
1,forrest gump (1994),4.292285
2,"shawshank redemption, the (1994)",4.596627


In [155]:
# rating이 높은 순에서 낮은 순으로 정렬
tr_df2_st = tr_df2.sort_values(by = ['rating'], ascending = False)
tr_df2_st

,title,rating
2,"shawshank redemption, the (1994)",4.596627
1,forrest gump (1994),4.292285
0,back to the future (1985),4.097967


결과적으로, 유추된 가설은 정답이 되었다.
- 'shawshank redemption, the (1994)'	4.596627
- 'back to the future (1985)'    4.097967
- ==> 'shawshank redemption, the (1994)'의 rating 값이 'back to the future (1985)' 보다 0.5점 정도 높다.
- ====> 따라서, 이 경우에는 장르보다 rating이 기여도에 큰 영향을 미쳤음을 알 수 있다.

# 회고

factors(차원 수)를 키울수록 값이 점점 높아짐, iteration(반복횟수)은 30 이상으로 키울수록 값이 점점 낮아짐.
- factors=100, regularization=0.01, use_gpu=False, iterations=15 ==> dreamb_vector, matrix_the_vector 0.4944174
- factors=100, regularization=0.01, use_gpu=False, iterations=30 ==> dreamb_vector, matrix_the_vector 0.52884734
- factors=150, regularization=0.01, use_gpu=False, iterations=30 ==> dreamb_vector, matrix_the_vector 0.6124598
- factors=200, regularization=0.01, use_gpu=False, iterations=30 ==> dreamb_vector, matrix_the_vector 0.6761505
- factors=250, regularization=0.01, use_gpu=False, iterations=30 ==> dreamb_vector, matrix_the_vector 0.75349844

기여도의 원인을 알아보는 과정에서 있었던 일
- 기여도의 원인을 알아내기 위해 기여한 영화들의 장르를 추출했다.
- 이 과정에서 대문자가 섞인 타이틀을 소문자로 변환하지 않고 삽질을 했다.
- 꼭 모두 소문자로 변환하고 작업할 것!